# Analysis Template

A template with the tools to get started pulling data from subgraphs or the blockchain.

In [4]:
import os
import asyncio
import requests
import pandas as pd
import json
from gql import Client, gql
from gql.transport.aiohttp import AIOHTTPTransport
from decimal import Decimal


In [5]:
## constants
INFURA_KEY = os.getenv('INFURA_KEY')

# mainnet
SUBGRAPH_ENDPOINT = 'https://api.thegraph.com/subgraphs/name/kwenta/optimism-main'
RPC_ENDPOINT = f'https://optimism-mainnet.infura.io/v3/{INFURA_KEY}'

# testnet
# SUBGRAPH_ENDPOINT = 'https://api.thegraph.com/subgraphs/name/kwenta/optimism-kovan-main'
# RPC_ENDPOINT = f'https://optimism-kovan.infura.io/v3/{INFURA_KEY}'


In [10]:
# functions
convertDecimals = lambda x: Decimal(x) / Decimal(10**18)

def clean_df(df, decimal_cols):
    for col in decimal_cols:
        if col in df.columns:
            df[col] = df[col].apply(convertDecimals)
        else:
            print(f"{col} not in DataFrame")
    return df

async def run_query(query, params, endpoint=SUBGRAPH_ENDPOINT):
    transport = AIOHTTPTransport(url=endpoint)

    async with Client(
        transport=transport,
        fetch_schema_from_transport=True,
    ) as session:

        # Execute single query
        query = query

        result = await session.execute(query, variable_values=params)
        df = pd.DataFrame(result)
        return df

async def run_recursive_query(query, params, accessor, endpoint=SUBGRAPH_ENDPOINT):
  transport = AIOHTTPTransport(url=endpoint)

  async with Client(
      transport=transport,
      fetch_schema_from_transport=True,
  ) as session:
    done_fetching = False
    all_results = []
    while not done_fetching:
      result = await session.execute(query, variable_values=params)
      if len(result[accessor]) > 0:
        all_results.extend(result[accessor])
        params['last_id'] = all_results[-1]['id']
      else:
        done_fetching = True
    
    df = pd.DataFrame(all_results)
    return df


In [11]:
# queries
marketsQuery = gql("""
query futuresMarkets(
    $last_id: ID!
  ) {
    futuresMarkets(
      where: {
        id_gt: $last_id
      }
      first: 1000
     ) {
      id
      asset
    }  
}
""")


### Run a query

In [13]:
market_params = {
    'last_id': ''
}

df_markets = await run_recursive_query(marketsQuery, market_params, 'futuresMarkets')
df_markets

,id,asset
0,0x001b7876f567f0b3a639332ed1e363839c6d85e2,0x41415645000000000000000000000000000000000000...
1,0x10305c1854d6db8a1060df60bdf8a8b2981249cf,0x44594458000000000000000000000000000000000000...
2,0x1228c7d8bbc5bc53db181bd7b1fce765aa83bf8a,0x734c494e4b0000000000000000000000000000000000...
3,0x3ed04ceff4c91872f19b1da35740c0be9ca21558,0x584d5200000000000000000000000000000000000000...
4,0x4434f56ddbde28fab08c4ae71970a06b300f8881,0x58415500000000000000000000000000000000000000...
5,0x4aa0dabd22bc0894975324bec293443c8538bd08,0x424e4200000000000000000000000000000000000000...
6,0x4ff54624d5fb61c34c634c3314ed3bfe4dbb665a,0x41564158000000000000000000000000000000000000...
7,0x5af0072617f7f2aeb0e314e2fad1de0231ba97cd,0x554e4900000000000000000000000000000000000000...
8,0x9f1c2f0071bc3b31447aeda9fa3a68d651eb4632,0x4f500000000000000000000000000000000000000000...
9,0x9f231dbe53d460f359b2b8cc47574493caa5b7bf,0x444f4745000000000000000000000000000000000000...
